In [1]:
from __future__ import print_function
import os

import numpy as np
import zipfile
import tarfile
from six.moves.urllib.request import urlretrieve
import shutil 
import random
import math

import string
import tensorflow as tf

# HYPERPARAMETERS.
# Size of batch
BATCH_SIZE = 1
# Length of sequence (=  number of units of controller (recurrent layer))
SEQ_LENGTH = 5

# Length of a vector - a single hidden state
HIDDEN_SIZE = 32

# Size of the input vector - 59 = [A-Z] + [a-z] + ' '
INPUT_SIZE = 59

#### MANN-related parameters.http://localhost:8888/notebooks/tensorflow-sandbox/ptb/pure_formulas/ptb_char_lstm_placeholder_memory.ipynb#
# Size of the local memory of each cell.
NUMBER_OF_SLOTS = 32
# Length of a single vector - a single memory slot.
ADDRESS_SIZE = 8
CONTENT_SIZE = 16
SLOT_SIZE = ADDRESS_SIZE + CONTENT_SIZE

# Dirs - must be absolute paths!
#LOG_DIR = '/tmp/tf/ptb_char_lstm_placeholder/B100S20_H64_N64A8C16/'
LOG_DIR = "/tmp/tf/ptb_char_lstm_ntm_memory2D/B"+\
    str(BATCH_SIZE)+"S"+str(SEQ_LENGTH)+"_H"+str(HIDDEN_SIZE)+"_N"+\
    str(NUMBER_OF_SLOTS)+"A"+str(ADDRESS_SIZE)+"C"+str(CONTENT_SIZE)+"_only_memory_output/"
print("Writing TB log to:",LOG_DIR)

# Local dir where PTB files will be stored.
PTB_DIR = "/home/tkornuta/data/ptb/"

# Filenames.
TRAIN = "ptb.train.txt"
VALID = "ptb.valid.txt"
TEST = "ptb.test.txt"



Writing TB log to: /tmp/tf/ptb_char_lstm_ntm/B1S5_H32_N32A8C16_only_memory_output/


### Check/maybe download PTB.

In [2]:
def maybe_download_ptb(path, 
                       filename='simple-examples.tgz', 
                       url='http://www.fit.vutbr.cz/~imikolov/rnnlm/', 
                       expected_bytes =34869662):
  # Eventually create the PTB dir.
  if not tf.gfile.Exists(path):
    tf.gfile.MakeDirs(path)
  """Download a file if not present, and make sure it's the right size."""
  _filename = path+filename
  if not os.path.exists(_filename):
    print('Downloading %s...' % filename)
    _filename, _ = urlretrieve(url+filename, _filename)
  statinfo = os.stat(_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', (_filename), '(', statinfo.st_size, ')')
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + _filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download_ptb(PTB_DIR)

Found and verified /home/tkornuta/data/ptb/simple-examples.tgz ( 34869662 )


### Extract dataset-related files from the PTB archive.

In [3]:
def extract_ptb(path, filename='simple-examples.tgz', files=["ptb.train.txt", "ptb.valid.txt", "ptb.test.txt", 
                                       "ptb.char.train.txt", "ptb.char.valid.txt", "ptb.char.test.txt"]):
    """Extracts files from PTB archive."""
    # Extract
    tar = tarfile.open(path+filename)
    tar.extractall(path)
    tar.close()
    # Copy files
    for file in files:
        shutil.copyfile(PTB_DIR+"simple-examples/data/"+file, PTB_DIR+file)
    # Delete directory
    shutil.rmtree(PTB_DIR+"simple-examples/")        

extract_ptb(PTB_DIR)

### Load train, valid and test texts.

In [4]:
def read_data(filename, path):
    with open(path+filename, 'r') as myfile:
        data=myfile.read()# .replace('\n', '')
        return data

train_text = read_data(TRAIN, PTB_DIR)
train_size=len(train_text)
print(train_size, train_text[:100])

valid_text = read_data(VALID, PTB_DIR)
valid_size=len(valid_text)
print(valid_size, valid_text[:64])

test_text = read_data(TEST, PTB_DIR)
test_size=len(test_text)
print(test_size, test_text[:64])

5101618  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote
399782  consumers may want to move their telephones a little closer to 
449945  no it was n't black monday 
 but while the new york stock excha


### Utility functions to map characters to vocabulary IDs and back.

In [5]:
first_letter = ord(string.ascii_uppercase[0]) # ascii_uppercase before lowercase! 
print("vocabulary size = ", INPUT_SIZE)
print(first_letter)

def char2id(char):
  """ Converts char to id (int) with one-hot encoding handling of unexpected characters"""
  if char in string.ascii_letters:# or char in string.punctuation or char in string.digits:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    # print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  """ Converts single id (int) to character"""
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

#print(len(string.punctuation))
#for i in string.ascii_letters:
#    print (i, char2id(i))


print(char2id('a'), char2id('A'), char2id('z'), char2id('Z'), char2id(' '), char2id('ï'))
print(id2char(char2id('a')), id2char(char2id('A')))
#print(id2char(65), id2char(33), id2char(90), id2char(58), id2char(0))
#bankno
sample = np.zeros(shape=(1, INPUT_SIZE), dtype=np.float)
sample[0, char2id(' ')] = 1.0
print(sample)

vocabulary size =  59
65
33 1 58 26 0 0
a A
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]


### Helper class for batch generation

In [6]:
class BatchGenerator(object):
  def __init__(self, text, batch_size, seq_length, vocab_size):
    """
    Initializes the batch generator object. Stores the variables and first "letter batch".
    text is text to be processed
    batch_size is size of batch (number of samples)
    seq_length represents the length of sequence
    vocab_size is number of words in vocabulary (assumes one-hot encoding)
    """
    # Store input parameters.
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._seq_length = seq_length
    self._vocab_size = vocab_size
    # Divide text into segments depending on number of batches, each segment determines a cursor position for a batch.
    segment = self._text_size // batch_size
    # Set initial cursor position.
    self._cursor = [ offset * segment for offset in range(batch_size)]
    # Store first "letter batch".
    self._last_letter_batch = self._next_letter_batch()
  
  def _next_letter_batch(self):
    """
    Returns a batch containing of encoded single letters depending on the current batch 
    cursor positions in the data.
    Returned "letter batch" is of size batch_size x vocab_size
    """
    letter_batch = np.zeros(shape=(self._batch_size, self._vocab_size), dtype=np.float)
    # Iterate through "samples"
    for b in range(self._batch_size):
      # Set 1 in position pointed out by one-hot char encoding.
      letter_batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return letter_batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    # First add last letter from previous batch (the "additional one").
    batches = [self._last_letter_batch]
    for step in range(self._seq_length):
      batches.append(self._next_letter_batch())
    # Store last "letter batch" for next batch.
    self._last_letter_batch = batches[-1]
    return batches


In [7]:
# Create objects for training, validation and testing batch generation.
train_batches = BatchGenerator(train_text, BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)

# For validation  - process the whole text as one big batch.
VALID_BATCH_SIZE = int(np.floor(valid_size/SEQ_LENGTH))
valid_batches = BatchGenerator(valid_text, VALID_BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)
# Get a single batch! 
valid_batch = valid_batches.next()

# For texting  - process the whole text as one big batch.
TEST_BATCH_SIZE = int(np.floor(test_size/SEQ_LENGTH))
test_batches = BatchGenerator(test_text, TEST_BATCH_SIZE, SEQ_LENGTH, INPUT_SIZE)
# Get a single batch! 
test_batch = test_batches.next()


### Helper functions - used during graph definition

In [8]:
# Function adding visualization to a given "matrix" (keys, memory etc.) with additional normalization
def visualize_hot_cold_normalized(matrix_,
                    axis_of_reduction_,
                    name_):
    with tf.name_scope(name_+"_vis_hc_norm"):
        # Eps for normalization in visualization
        EPS = 1e-10
        
        # Create hot-cold visualization (red=positive/blue=negative)
        zeros = tf.zeros_like(matrix_) 
        
        # Get negative values only.
        neg = tf.less(matrix_, zeros)
        blue = tf.multiply(tf.cast(neg, tf.float32), matrix_)
        min_blue = tf.reduce_min(matrix_, axis=axis_of_reduction_, keep_dims=True) + EPS
        norm_blue = 255.0 * blue/min_blue
        
        # Get positive values only.
        pos = tf.greater(matrix_, zeros)
        red = tf.multiply(tf.cast(pos, tf.float32), matrix_)
        max_red = tf.reduce_max(matrix_, axis=axis_of_reduction_, keep_dims=True) + EPS
        norm_red = 255.0 * red/max_red
        
        # Stack them into three channel image with hot-cold values.
        rgb = tf.stack([norm_red, zeros, norm_blue], axis=2)

        #print("name_=",name_)
        #print("np.int32(zeros.shape[1]=", np.int32(zeros.shape[1]))
        # TODO: find fix and get rid of the batch_size(!)
        rgb_reshaped = tf.reshape(rgb, [1, -1,  np.int32(zeros.shape[1]), 3])

        # Visualize read weights as image.
        rgb_reshaped_summary = tf.summary.image(name_+"_visv", rgb_reshaped)

In [9]:
# Function adding visualization to a given "matrix" (keys, memory etc.)
def visualize_hot_cold(matrix_,
                    name_):
    with tf.name_scope(name_+"_vis_hc"):
        # Eps for normalization in visualization
        EPS = 1e-10
        # Create hot-cold visualization (red=positive/blue=negative)
        zeros = tf.zeros_like(matrix_) 
        
        # Get negative values only.
        neg = tf.less(matrix_, zeros)
        blue = 255.0 * tf.multiply(tf.cast(neg, tf.float32), matrix_)
        
        # Get positive values only.
        pos = tf.greater(matrix_, zeros)
        red = 255.0 * tf.multiply(tf.cast(pos, tf.float32), matrix_)
        
        # Stack them into three channel image with hot-cold values.
        rgb = tf.stack([red, zeros, blue], axis=2)

        #print("name_=",name_)
        #print("np.int32(zeros.shape[1]=", np.int32(zeros.shape[1]))
        # TODO: find fix and get rid of the batch_size(!)
        rgb_reshaped = tf.reshape(rgb, [1, -1,  np.int32(zeros.shape[1]), 3])

        # Visualize read weights as image.
        rgb_reshaped_summary = tf.summary.image(name_+"_vis", rgb_reshaped)


In [10]:
# Function "fixing" tensordot lack of dimensions.
def tensordot_fix(matrix_a_, matrix_b_, axes_, output_shape_):
    tensor_without_dims = tf.tensordot(matrix_a_, matrix_b_, axes=axes_)
    # TF bugfix - tensordot not returning proper shapes when only partial shapes are known. :]
    # https://github.com/tensorflow/tensorflow/issues/6682
    tensor = tf.reshape(tensor_without_dims, output_shape_)
    return tensor 
    

In [11]:
# Additional ops used by NTM.
def build_focusing_by_location(keys, beta, memory):
    """Computes content addressing. Uses both address and content part for calculation of the similarity.
    Args:
        key: a 2-D Tensor [BATCH_SIZE x SLOT_SIZE] 
        beta: a 1-D Tensor - key strength [BATCH_SIZE x 1]
        memory: a 2-D Tensor [SLOT_SIZE x NUMBER_OF_SLOTS]
    """
    with tf.name_scope("focusing_by_location"):
    
        # Normalize batch - along samples.
        norm_keys = tf.nn.l2_normalize(keys,1, name="normalized_keys")
        # Normalize memory - along slots 
        norm_memory = tf.nn.l2_normalize(memory, 0)

        # Calculate cosine similarity [BATCH_SIZE x NUMBER_OF_SLOTS].
        similarity = tf.matmul(norm_keys, norm_memory, name="similarity")

        # Element-wise multiplication [BATCH_SIZE x NUMBER_OF_SLOTS]
        strengthtened_similarity = beta * similarity

        # Calculate weighting based on similarity along the "slot dimension" [BATCH_SIZE x NUMBER_OF_SLOTS].
        result = tf.nn.softmax(strengthtened_similarity, dim=1)
        return result


def circular_index(idx, size):
    if idx < 0: return size + idx
    if idx >= size : return idx - size
    else: return idx

def build_circular_convolution(batch, kernel):
    """Computes circular convolution.
    Args:
        batch: a 2-D Tensor [BATCH_SIZE x NUMBER_OF_SLOTS] 
        kernel: a 2-D Tensor [BATCH_SIZE x KERNEL_SIZE (e.g. 3)]
    """
    with tf.name_scope("circular_convolution"):
        size = int(batch.get_shape()[1])
        kernel_size = int(kernel.get_shape()[1])
        kernel_shift = int(math.floor(kernel_size/2.0))

        kernels = []
        for i in range(size):
            # Create a list of index vectors.
            indices = [circular_index(i+j, size) for j in range(kernel_shift, -kernel_shift-1, -1)]
            # Reorganize batch according to indices. 
            reorganized_batch = tf.gather(batch, indices, axis=1)
            # Perform convolution.
            kernels.append(tf.reduce_sum(reorganized_batch * kernel, 1))
            
        # Sum elements lying on the same positions.
        result_without_dims = tf.transpose(tf.dynamic_stitch([i for i in range(size)], kernels))
        result_BxN = tf.reshape(result_without_dims, [-1, size])
        return result_BxN


def build_sharpening(batch, gamma):
    """Computes sharpening.
    Args:
        batch: a 2-D Tensor [BATCH_SIZE x NUMBER_OF_SLOTS] 
        gamma: a 1-D Tensor [BATCH_SIZE x 1]
    """
    EPS = 1e-30
    with tf.name_scope("sharpening"):    
        number_of_slots = int(batch.get_shape()[1])

        # Duplicate gammas - tf.tile is not working for partially unknown shape :] 
        gammas = []
        for i in range(number_of_slots):
            # Truncates gamma to 50!
            gammas.append(tf.minimum(gamma[:,0], 50))
        gammas_stacked = tf.transpose(tf.stack(gammas))
        # Calculate powered batch [BATCH_SIZE x NUMBER_OF_SLOTS].
        powed_batch = tf.pow(batch, gammas_stacked)+EPS

        # "Normalization" [BATCH_SIZE x NUMBER_OF_SLOTS].
        sharpened_batch = (powed_batch) / (tf.reduce_sum(powed_batch, axis=1, keep_dims=True))

        return sharpened_batch


In [12]:
def build_head(keys_BxS, beta_Bx1, memory_SxN, interpolation_gate_Bx1, prev_weights_BxN, shift_Bx3, gamma_Bx1):
    # 1. Content addressing  [BATCH_SIZE x NUMBER_OF_SLOTS].
    content_weights_BxN = build_focusing_by_location(keys_BxS, beta_Bx1, memory_SxN)

    # 2. Perform "gated interpolation" [BATCH_SIZE x NUMBER_OF_SLOTS].
    gated_weights_BxN = tf.add(interpolation_gate_Bx1 * content_weights_BxN,
        (1 - interpolation_gate_Bx1) * prev_weights_BxN,
        name="gated_weights_BxN")

    # 3. Shift  weights [BATCH_SIZE x NUMBER_OF_SLOTS].
    shifted_weights_BxN = build_circular_convolution(gated_weights_BxN, shift_Bx3)

    # 4. "Sharpen" the weights [BATCH_SIZE x NUMBER_OF_SLOTS].
    sharpened_weights_BxN = build_sharpening(shifted_weights_BxN, gamma_Bx1)

    # Add histograms to TensorBoard.
    tf.summary.histogram("1.content_weights_BxN", content_weights_BxN)
    tf.summary.histogram("2.gated_weights_BxN", gated_weights_BxN)
    tf.summary.histogram("3.shifted_weights_BxN", shifted_weights_BxN)
    tf.summary.histogram("4.sharpened_weights_BxN", sharpened_weights_BxN)
    # Add hot-cold visualizations.
    visualize_hot_cold(matrix_=content_weights_BxN, name_="1.content_weights_BxN")          
    visualize_hot_cold(matrix_=gated_weights_BxN, name_="2.gated_weights_BxN")          
    visualize_hot_cold(matrix_=shifted_weights_BxN, name_="3.shifted_weights_BxN")          
    visualize_hot_cold(matrix_=sharpened_weights_BxN, name_="4.sharpened_weights_BxN")          

    return sharpened_weights_BxN



In [13]:
# Definition of graph of the MANN controller cell.
def controller_cell(input_BxI, # input x [BATCH_SIZE x INPUT_SIZE]
                    prev_cell_state_BxH, # BATCH x HIDDEN
                    prev_cell_output_BxH, # BATCH X HIDDEN
                    prev_memory_output_BxS, # read vector from the memory returned by previous cell [BATCH_SIZE x SLOT_SIZE]
                    prev_memory_SxN, # Value of the memory from previous time state [SLOT_SIZE, NUMBER_OF_SLOTS]
                    prev_read_weights_BxN, # read weights from previous time state (t-1) [BATCH_SIZExNUMBER_OF_SLOTS]
                    prev_write_weights_BxN, # read weights from previous time state (t-1) [BATCH_SIZExNUMBER_OF_SLOTS]
                    name):
    """Creates a controller cell with shared memory"""
    #with tf.name_scope(name):
    #######################################################################################
    # A) Get LSTM shared variables.   
    #with tf.variable_scope("lstm_variables_aaa", reuse=True):

        # Input gate: input, previous output, and bias.
    #    Wf = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wf")
    #    bf = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bf")

        # Forget gate: input, previous output, and bias.
    #    Wi = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wi")
    #    bi = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bi")

        # Memory cell: input, state and bias.                             
    #    Wc = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wc")
    #    bc = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bc")

        # Output gate: input, previous output, and bias.
    #    Wo = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wo")
    #    bo = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bo")
        # Add histograms to TensorBoard.
    #    tf.summary.histogram("Wf", Wf)
    #    tf.summary.histogram("Wi", Wi)
    #    tf.summary.histogram("Wc", Wc)
    #    tf.summary.histogram("Wo", Wo)
    
    with tf.name_scope("lstm"):
        #######################################################################################
        # B) Calculate LSTM hidden state and output.

        # Concatenate h_prev ("prev output") with x.
        x_prev_h = tf.concat([input_BxI, prev_cell_output_BxH], 1)

        # Calculate forget, input and output gates activations.
        forget_gate = tf.sigmoid(tf.matmul(x_prev_h, Wf) + bf, name="forget_gate")
        input_gate = tf.sigmoid(tf.matmul(x_prev_h, Wi) + bi, name="Input_gate")
        output_gate = tf.sigmoid(tf.matmul(x_prev_h, Wo) + bo, name="Output_gate")

        # Calcualte cell state update (C~).
        cell_update = tf.tanh(tf.matmul(x_prev_h, Wc) + bc, name="Cell_update")
        # New cell state (C).
        cell_state_BxH = tf.add(forget_gate * prev_cell_state_BxH, input_gate * cell_update, name = "Cell_state")

        # Calculate cell output (h).
        cell_output_BxH = output_gate * tf.tanh(cell_state_BxH)

        # Add histograms to TensorBoard.
        tf.summary.histogram("input_BxI", input_BxI)
        tf.summary.histogram("prev_cell_state_BxH", prev_cell_state_BxH)
        tf.summary.histogram("prev_cell_output_BxH", prev_cell_output_BxH)
        tf.summary.histogram("prev_memory_output_BxS", prev_memory_output_BxS)
        tf.summary.histogram("cell_state_BxH", cell_state_BxH)
        tf.summary.histogram("cell_output_BxH", cell_output_BxH)

        # Add hot-cold visualizations.
        visualize_hot_cold_normalized(matrix_=input_BxI, axis_of_reduction_=None, name_="input_BxI")
        visualize_hot_cold_normalized(matrix_=prev_cell_state_BxH, axis_of_reduction_=None, name_="prev_cell_state_BxH")
        visualize_hot_cold_normalized(matrix_=prev_cell_output_BxH, axis_of_reduction_=None, name_="prev_cell_output_BxH")
        visualize_hot_cold_normalized(matrix_=prev_memory_output_BxS, axis_of_reduction_=None, name_="prev_memory_output_BxS")
        visualize_hot_cold_normalized(matrix_=cell_state_BxH, axis_of_reduction_=None, name_="cell_state_BxH")      
        visualize_hot_cold_normalized(matrix_=cell_output_BxH, axis_of_reduction_=None, name_="cell_output_BxH")      

    #######################################################################################
    # A) Get shared variables.   
    #with tf.variable_scope("read_head_variables_aaa", reuse=True):
        # 1. Content addressing.
    #    Wh_rk = tf.get_variable("Wh_rk", [HIDDEN_SIZE, SLOT_SIZE])
    #    Wh_rbeta = tf.get_variable("Wh_rbeta", [HIDDEN_SIZE, 1])

        # 2. Interpolation gate.
    #    Wh_rig = tf.get_variable("Wh_rig", [HIDDEN_SIZE, 1])

        # 3. Shift.
    #    Wh_rsh = tf.get_variable("Wh_rsh", [HIDDEN_SIZE, 3])

        # 4. Sharpening.
    #    Wh_rgamma = tf.get_variable("Wh_rgamma", [HIDDEN_SIZE, 1])

    # Build the read head.
    with tf.name_scope("read_head"): 
        #######################################################################################
        # B) "Emit" parameters on the basis of controller output.       
        # 1. Content addressing
        # Calculate read keys [BATCH_SIZE x SLOT_SIZE] range: -inf - +inf!
        rkey_BxS = tf.nn.relu(tf.matmul(cell_output_BxH, Wh_rk, name="key_BxS"))
        # Calculate betas - used for sharpening/smoothing similarity - scalar [BATCH x 1] range: +1 - +inf!
        rbeta_Bx1 = 5 + tf.log1p(tf.exp(tf.matmul(cell_output_BxH, Wh_rbeta)), name="beta_Bx1")

        # 2. Focusing by location - interpolation gate - [BATCH_SIZE x 1] - range: 0 - +1.
        rinterpolation_gate_Bx1 = tf.sigmoid(tf.matmul(cell_output_BxH, Wh_rig), name="interpolation_gate_Bx1")

        # 3. Shift weighting - for circular convolution [BATCH_SIZE x 3] - range: 0 - +1.
        rshift_Bx3 = tf.sigmoid(tf.matmul(cell_output_BxH, Wh_rsh), name="shift_Bx3")

        # 4. Sharpening
        # Gamma - scalar [BATCH x 1] range: 1 - +inf
        rgamma_Bx1 = 5 + tf.log1p(tf.exp(tf.matmul(cell_output_BxH, Wh_rgamma)))

        #######################################################################################
        # C) Build the read head! [BATCH_SIZE x NUMBER_OF_SLOTS]
        read_weights_BxN = build_head(rkey_BxS, rbeta_Bx1, prev_memory_SxN, rinterpolation_gate_Bx1, prev_read_weights_BxN, rshift_Bx3, rgamma_Bx1)
        print("read_weights_BxN=",read_weights_BxN)

         # Add histograms to TensorBoard.
        tf.summary.histogram("7.read_weights_BxN", read_weights_BxN)
        # Add hot-cold visualizations.
        visualize_hot_cold(matrix_=read_weights_BxN, name_="7.read_weights_BxN")          

    # Read from the memory.
    with tf.name_scope("memory_output"):
        # Calculate the memory output ("read vector").
        memory_output_without_dims = tf.tensordot(read_weights_BxN, tf.transpose(prev_memory_SxN), axes=1, name="memory_output_without_dims")  
        # TF bugfix - tensordot not returning proper shapes when only partial shapes are known. :]
        # https://github.com/tensorflow/tensorflow/issues/6682
        memory_output_BxS = tf.reshape(memory_output_without_dims, [-1, SLOT_SIZE], name="memory_output_BxS")

        # Add histograms to TensorBoard.
        tf.summary.histogram("8.memory_output_BxS", memory_output_BxS)

        # Add hot-cold visualizations.
        visualize_hot_cold_normalized(matrix_=memory_output_BxS, axis_of_reduction_=None, name_="8.memory_output_BxS")          


    #######################################################################################
    # A) Get shared variables.   
    #with tf.variable_scope("write_head_variables_aaa", reuse=True):
        # 1. Content addressing.
    #    Wh_wk = tf.get_variable("Wh_wk", [HIDDEN_SIZE, SLOT_SIZE])
    #    Wh_wbeta = tf.get_variable("Wh_wbeta", [HIDDEN_SIZE, 1])

        # 2. Interpolation gate.
    #    Wh_wig = tf.get_variable("Wh_wig", [HIDDEN_SIZE, 1])

        # 3. Shift.
    #    Wh_wsh = tf.get_variable("Wh_wsh", [HIDDEN_SIZE, 3])

        # 4. Sharpening.
    #    Wh_wgamma = tf.get_variable("Wh_wgamma", [HIDDEN_SIZE, 1])
        
    # Build the write head.
    with tf.name_scope("write_head"): 
        #######################################################################################
        # B) "Emit" parameters on the basis of controller output.       
        # 1. Content addressing
        # Calculate write keys [BATCH_SIZE x SLOT_SIZE] range: -inf - +inf!
        wkey_BxS = tf.nn.relu(tf.matmul(cell_output_BxH, Wh_wk, name="key_BxS"))
        # Calculate betas - used for sharpening/smoothing similarity - scalar [BATCH x 1] range: +1 - +inf!
        wbeta_Bx1 = 5 + tf.log1p(tf.exp(tf.matmul(cell_output_BxH, Wh_wbeta)), name="beta_Bx1")

        # 2. Focusing by location - interpolation gate - [BATCH_SIZE x 1] - range: 0 - +1.
        winterpolation_gate_Bx1 = tf.sigmoid(tf.matmul(cell_output_BxH, Wh_wig), name="interpolation_gate_Bx1")

        # 3. Shift weighting - for circular convolution [BATCH_SIZE x 3] - range: 0 - +1.
        wshift_Bx3 = tf.sigmoid(tf.matmul(cell_output_BxH, Wh_wsh), name="shift_Bx3")

        # 4. Sharpening
        # Gamma - scalar [BATCH x 1] range: 1 - +inf!
        wgamma_Bx1 = 5 + tf.log1p(tf.exp(tf.matmul(cell_output_BxH, Wh_wgamma)))

        #######################################################################################
        # C) Build the read head! [BATCH_SIZE x NUMBER_OF_SLOTS]
        write_weights_BxN = build_head(wkey_BxS, wbeta_Bx1, prev_memory_SxN, winterpolation_gate_Bx1, prev_read_weights_BxN, wshift_Bx3, wgamma_Bx1)
        print("write_weights_BxN=",write_weights_BxN)

         # Add histograms to TensorBoard.
        tf.summary.histogram("7.write_weights_BxN", write_weights_BxN)
        # Add hot-cold visualizations.
        visualize_hot_cold(matrix_=write_weights_BxN, name_="7.write_weights_BxN")          

    with tf.name_scope("memory_update"):
        #######################################################################################
        # A) Get shared variables.   
        with tf.variable_scope("memory_update_variables", reuse=True):
            # Add vector
            Wh_wa = tf.get_variable("Wh_wa", [HIDDEN_SIZE, CONTENT_SIZE])
            # Erase vector
            Wh_we = tf.get_variable("Wh_we", [HIDDEN_SIZE, CONTENT_SIZE])

            # Add histograms to TensorBoard.
            tf.summary.histogram("Wh_wa", Wh_wa)
            tf.summary.histogram("Wh_we", Wh_we)

        #######################################################################################
        # B) "Emit" parameters on the basis of controller output.       
        # Add vector [BATCH_SIZE x CONTENT_SIZE] range: -inf - +inf!
        add_vector_BxC = tf.nn.relu(tf.matmul(cell_output_BxH, Wh_wa, name="add_vector_BxC"))

        # Erase vector [BATCH_SIZE x CONTENT_SIZE] range: 0 - +1
        erase_vector_BxC = 0.3 * tf.sigmoid(tf.matmul(cell_output_BxH, Wh_we, name="erase_vector_BxS"))

        #0. Get content part of the memory.
        #content = tf.slice(prev_memory_, [ADDRESS_SIZE, 0], [CONTENT_SIZE, NUMBER_OF_SLOTS], name="memory_content") 

        # 3. Preservation gate p [CONTENT_SIZE x NUMBER_OF_SLOTS]
        preserved_content_CxN = tf.ones([CONTENT_SIZE, NUMBER_OF_SLOTS]) - tf.matmul(tf.transpose(erase_vector_BxC), write_weights_BxN)
        # How much memory will "preserve" [SLOT_SIZE x NUMBER_OF_SLOTS] 
        preserved_memory_SxN = prev_memory_SxN * tf.concat(
            [tf.ones([ADDRESS_SIZE, NUMBER_OF_SLOTS], tf.float32), preserved_content_CxN],
            axis=0)

        # 4. Calculate update of the content [CONTENT_SIZE x NUMBER_OF_SLOTS]
        content_update_CxN = tf.matmul(tf.transpose(add_vector_BxC), write_weights_BxN, name="content_update")

        # Calculate update of memory [SLOT_SIZE x NUMBER_OF_SLOTS]
        memory_update_SxN = tf.concat(
            [tf.zeros([ADDRESS_SIZE, NUMBER_OF_SLOTS], tf.float32), content_update_CxN],
            axis=0, name="memory_update")

        # 5. Calculate "updated" memory [SLOT_SIZE x MEMORY_SIZE]
        #memory_updated_SxN = prev_memory_SxN          
        memory_updated_SxN = tf.clip_by_value(preserved_memory_SxN + memory_update_SxN, 0.0, 1.0)          
        #memory_updated = tf.clip_by_value(memory_preserved + update, 0.0, 1.0)
        #memory_updated = tf.tanh(memory_preserved + update)

        # Add histograms to TensorBoard.
        tf.summary.histogram("1.add_vector_BxC", add_vector_BxC)
        tf.summary.histogram("2.erase_vector_BxC", erase_vector_BxC)
        tf.summary.histogram("3.preserved_content_CxN", preserved_content_CxN)
        tf.summary.histogram("4.content_update_CxN", content_update_CxN)
        tf.summary.histogram("5.memory_updated_SxN", memory_updated_SxN)
        tf.summary.histogram("0.prev_memory_SxN", prev_memory_SxN)

        # Add hot-cold visualizations.
        visualize_hot_cold_normalized(matrix_=add_vector_BxC, axis_of_reduction_=None, name_="1.add_vector_BxC")          
        visualize_hot_cold_normalized(matrix_=erase_vector_BxC, axis_of_reduction_=None, name_="2.erase_vector_BxC")          
        visualize_hot_cold_normalized(matrix_=preserved_content_CxN, axis_of_reduction_=None, name_="3.preserved_content_CxN")          
        visualize_hot_cold_normalized(matrix_=content_update_CxN, axis_of_reduction_=None, name_="4.content_update_CxN")          
        visualize_hot_cold_normalized(matrix_=memory_updated_SxN, axis_of_reduction_=None, name_="5.memory_updated_SxN")          
        visualize_hot_cold_normalized(matrix_=prev_memory_SxN, axis_of_reduction_=None, name_="0.prev_memory_SxN")          

    return cell_state_BxH, cell_output_BxH, memory_output_BxS, memory_updated_SxN, read_weights_BxN, read_weights_BxN # FOR NOW!!

print("Cell definition OK")

Cell definition OK


## Built graph

In [14]:
# Reset graph - just in case.
tf.reset_default_graph()

# Create shared (reuseable) space for all cells.   
with tf.variable_scope("lstm_variables"):
    # Input gate: input, previous output, and bias.
    Wf = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wf")
    bf = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bf")

    # Forget gate: input, previous output, and bias.
    Wi = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wi")
    bi = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bi")

    # Memory cell: input, state and bias.                             
    Wc = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wc")
    bc = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bc")

    # Output gate: input, previous output, and bias.
    Wo = tf.Variable(tf.truncated_normal([INPUT_SIZE+HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="Wo")
    bo = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bo")
    
    # Initialization placeholders.
    init_cell_state = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_cell_state")
    init_cell_output = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_cell_output")

    # Add histograms to TensorBoard.
    tf.summary.histogram("Wf", Wf)
    tf.summary.histogram("Wi", Wi)
    tf.summary.histogram("Wc", Wc)
    tf.summary.histogram("Wo", Wo)
    
# Create shared (reuseable) space for all cells.   
with tf.variable_scope("read_head_variables"):
    # 1. Content addressing.
    Wh_rk = tf.get_variable("Wh_rk", [HIDDEN_SIZE, SLOT_SIZE])
    Wh_rbeta = tf.get_variable("Wh_rbeta", [HIDDEN_SIZE, 1])

    # 2. Interpolation gate.
    Wh_rig = tf.get_variable("Wh_rig", [HIDDEN_SIZE, 1])

    # 3. Shift.
    Wh_rsh = tf.get_variable("Wh_rsh", [HIDDEN_SIZE, 3])

    # 4. Sharpening.
    Wh_rgamma = tf.get_variable("Wh_rgamma", [HIDDEN_SIZE, 1])

    # Add histograms to TensorBoard.
    tf.summary.histogram("Wh_rk", Wh_rk)
    tf.summary.histogram("Wh_rbeta", Wh_rbeta)
    tf.summary.histogram("Wh_rig", Wh_rig)
    tf.summary.histogram("Wh_rsh", Wh_rsh)
    tf.summary.histogram("Wh_rgamma", Wh_rgamma)
    
with tf.variable_scope("write_head_variables"):
    # 1. Content addressing.
    Wh_wk = tf.get_variable("Wh_wk", [HIDDEN_SIZE, SLOT_SIZE])
    Wh_wbeta = tf.get_variable("Wh_wbeta", [HIDDEN_SIZE, 1])

    # 2. Interpolation gate.
    Wh_wig = tf.get_variable("Wh_wig", [HIDDEN_SIZE, 1])

    # 3. Shift.
    Wh_wsh = tf.get_variable("Wh_wsh", [HIDDEN_SIZE, 3])

    # 4. Sharpening.
    Wh_wgamma = tf.get_variable("Wh_wgamma", [HIDDEN_SIZE, 1])

    # Add histograms to TensorBoard.
    tf.summary.histogram("Wh_wk", Wh_wk)
    tf.summary.histogram("Wh_wbeta", Wh_wbeta)
    tf.summary.histogram("Wh_wig", Wh_wig)
    tf.summary.histogram("Wh_wsh", Wh_wsh)
    tf.summary.histogram("Wh_wgamma", Wh_wgamma)

# Memory initialization related ops.
with tf.name_scope("memory_initialization"):
    #memory = tf.get_variable("memory", tf.truncated_normal(shape=[HIDDEN_SIZE, NUMBER_OF_SLOTS]), trainable=False)
    init_memory_state_op = tf.truncated_normal(shape=[SLOT_SIZE, NUMBER_OF_SLOTS], mean=0.5, stddev=0.2, dtype=tf.float32)
    #init_memory_state_op = tf.clip_by_value(tf.truncated_normal(shape=[SLOT_SIZE, NUMBER_OF_SLOTS], dtype=tf.float32), 0.0, 1.0)

    # Sets sparse memory addresses.
    #memory_content = tf.slice(init_memory_state_op, [ADDRESS_SIZE, 0], [CONTENT_SIZE, NUMBER_OF_SLOTS])
    #sparse_address = tf.eye(ADDRESS_SIZE) # must be = NUMBER_OF_SLOTS!
    #init_sparse_addressing_memory_state_op = tf.concat([sparse_address, memory_content], axis = 0)

    # Placeholder for "initial" memory output [BATCH_SIZE X SLOT_SIZE].
    init_memory_output = tf.placeholder(tf.float32, shape=[None, SLOT_SIZE], name="init_memory_read")
    input_memory_state = tf.placeholder(tf.float32, shape=[SLOT_SIZE, NUMBER_OF_SLOTS], name="input_memory_state")
    
with tf.variable_scope("memory_update_variables"):
    # Add vector
    Wh_wa = tf.get_variable("Wh_wa", [HIDDEN_SIZE, CONTENT_SIZE])
    # Erase vector
    Wh_we = tf.get_variable("Wh_we", [HIDDEN_SIZE, CONTENT_SIZE])

    # Add histograms to TensorBoard.
    tf.summary.histogram("Wh_wa", Wh_wa)
    tf.summary.histogram("Wh_we", Wh_we)

# 0. Previous variables.
with tf.name_scope("Previous_variables"):

    # Placeholders for previous weights.
    # TODO: Each cell (element in sequence) has its own prev read/update vector - ONE FOR THE WHOLE BATCH [NUMBER_OF_SLOTS]
    prev_read_weights_batch_seq = list()    
    for i in range(SEQ_LENGTH):
        prev_read_weights_batch_seq.append(tf.placeholder(tf.float32, shape=[None, NUMBER_OF_SLOTS], name="prev_read_weights_BxN"+str(i)))
    prev_write_weights_batch_seq = list()    
    for i in range(SEQ_LENGTH):
        prev_write_weights_batch_seq.append(tf.placeholder(tf.float32, shape=[None, NUMBER_OF_SLOTS], name="prev_write_weights_BxN"+str(i)))

# 1. Placeholders for inputs.
with tf.name_scope("input_data"):
    # Define input data buffers.
    data_buffers = list()
    for i in range(SEQ_LENGTH + 1):
        # Collect placeholders for inputs/labels: Batch x Vocab size.
        data_buffers.append(tf.placeholder(tf.float32, shape=[None, INPUT_SIZE], name="data_buffers_BxI_"+str(i)))

    # Sequence of batches.
    input_BxI_L = data_buffers[:SEQ_LENGTH]

    # Labels are pointing to the same placeholders!
    # Labels are inputs shifted by one time step.
    target_BxI_L = data_buffers[1:]  
    # Concatenate targets into 2D tensor.
    target_LBxI = tf.concat(target_BxI_L, 0)

    # Add histograms to TensorBoard.
    tf.summary.histogram("input_BxI_L", input_BxI_L)


# Build list of outpus for sequence (at the end of size SEQ_LENGTH).
memory_outputs_batch_seq = list()
cell_output_batch_seq = list()
# Two lists that will be "returned" and later passed as previous states. 
read_weights_batch_seq = list()  
write_weights_batch_seq = list()  
intermediate_memory_states = list()

# Build "unrolled controller" - "link" oldest memory output to initial placeholder.
print("Building NTM cells - it might take a while...")
cell_state = init_cell_state 
cell_output = init_cell_output
memory_output = init_memory_output
memory_state = input_memory_state
# For every buffer in input sequence batch buffers...
for i in range(SEQ_LENGTH):
    # ... add cell...     
    cell_state, cell_output, memory_output, memory_state, read_weights, write_weights = controller_cell(
        input_BxI_L[i], 
        cell_state,
        cell_output,
        memory_output,
        memory_state,
        prev_read_weights_batch_seq[i],
        prev_write_weights_batch_seq[i],
        "cell_"+str(i))
    # .. and store outputs and read/write weights.
    memory_outputs_batch_seq.append(memory_output)
    cell_output_batch_seq.append(cell_output)
    read_weights_batch_seq.append(read_weights)
    write_weights_batch_seq.append(write_weights)
    intermediate_memory_states.append(memory_state)

with tf.name_scope("memory_state"):
    # Final memory state.
    final_memory_state = memory_state

    # Add histograms to TensorBoard.
    tf.summary.histogram("input_memory_state_SxN", input_memory_state)
    tf.summary.histogram("final_memory_state_SxN", final_memory_state)
    visualize_hot_cold_normalized(matrix_=input_memory_state, axis_of_reduction_=None, name_="input_memory_state_SxN")          
    visualize_hot_cold_normalized(matrix_=final_memory_state, axis_of_reduction_=None, name_="final_memory_state_SxN")          
    for i in range(SEQ_LENGTH):
        tf.summary.histogram("intermediate_memory_state_SxN_"+str(i), intermediate_memory_states[i])
        visualize_hot_cold_normalized(matrix_=intermediate_memory_states[i], axis_of_reduction_=None, name_="intermediate_memory_state_SxN_"+str(i))          

print("Building output...")
with tf.variable_scope("output_variables"):
    # Cell output weights and biases.
    Wcol = tf.get_variable("Wcol", [HIDDEN_SIZE, INPUT_SIZE])
    bcol = tf.get_variable("bcol", [INPUT_SIZE])
    # Memory output weights and biases.
    Wmol = tf.get_variable("Wmol", [SLOT_SIZE, INPUT_SIZE])
    bmol = tf.get_variable("bmol", [INPUT_SIZE])
    
# 3. Output ops.
with tf.name_scope("output"):
    # Concatenate cell output sequence along time axis.
    cell_output_batch = tf.concat(cell_output_batch_seq, 0) 
    # Memory logits.
    cell_logits_batch = tf.nn.xw_plus_b(cell_output_batch, Wcol, bcol, name = "cell_logits_batch")

    # Concatenate memory output sequence along time axis.
    memory_output_batch = tf.concat(memory_outputs_batch_seq, 0) 
    # Memory logits.
    memory_logits_batch = tf.nn.xw_plus_b(memory_output_batch, Wmol, bmol, name = "memory_logits_batch")
    
    # Add fully connected softmax layer on top.
    logits_batch = memory_logits_batch #+ cell_logits_batch
    prediction_batch = tf.nn.softmax(logits_batch)

 
# 4. Loss ops.
with tf.name_scope("loss"):
    # Loss function(s) - reduce accross every batch and sequence (output generated by every LSTM cell).
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_LBxI, logits=logits_batch))
    # Add loss summary.
    loss_summary = tf.summary.scalar("loss", loss)

# 5. Training ops.  
with tf.name_scope("optimization"):
    # Learning rate decay.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.1, global_step, 5000, 0.9, staircase=True)
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)
    gradients, v = zip(*grads_and_vars)
    # Gradient clipping.
    gradients, _ = tf.clip_by_global_norm(gradients, 0.1)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    print("Building summaries for all variables and gradients - it might take a while...")
    for i, (grad, var) in enumerate(grads_and_vars):
        if grad is not None:
            gradients[i] = (tf.clip_by_value(grad, -10, 10), var)
            tf.summary.histogram(var.name, var)
            tf.summary.histogram(var.name + '/grad', grad)
    
    
# Merge all summaries.
merged_summaries = tf.summary.merge_all()

print("Graph definition OK")

Building NTM cells - it might take a while...
read_weights_BxN= Tensor("read_head/sharpening/truediv:0", shape=(?, 32), dtype=float32)
write_weights_BxN= Tensor("write_head/sharpening/truediv:0", shape=(?, 32), dtype=float32)
read_weights_BxN= Tensor("read_head_1/sharpening/truediv:0", shape=(?, 32), dtype=float32)
write_weights_BxN= Tensor("write_head_1/sharpening/truediv:0", shape=(?, 32), dtype=float32)
read_weights_BxN= Tensor("read_head_2/sharpening/truediv:0", shape=(?, 32), dtype=float32)
write_weights_BxN= Tensor("write_head_2/sharpening/truediv:0", shape=(?, 32), dtype=float32)
read_weights_BxN= Tensor("read_head_3/sharpening/truediv:0", shape=(?, 32), dtype=float32)
write_weights_BxN= Tensor("write_head_3/sharpening/truediv:0", shape=(?, 32), dtype=float32)
read_weights_BxN= Tensor("read_head_4/sharpening/truediv:0", shape=(?, 32), dtype=float32)
write_weights_BxN= Tensor("write_head_4/sharpening/truediv:0", shape=(?, 32), dtype=float32)
Building output...
Building summaries 

###  Definition of tensor graph

In [15]:
def create_feed_dict(set_type_):
    """Creates feed dictionaries and set initial values for placeholders for different sets"""
    feed_dict = dict()
    
    if set_type_=="train":
        # Get next batch and create a feed dict.
        next_batch = train_batches.next()
        # Feed batch to input buffers.
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = next_batch[i]

        # Set previous weights of read and write heades.
        for i in range(SEQ_LENGTH):
            feed_dict[prev_read_weights_batch_seq[i]] = prev_rw_batch_seq[i]
            feed_dict[prev_write_weights_batch_seq[i]] = prev_ww_batch_seq[i]

        # Reset "initial" memory output.
        feed_dict[init_cell_state] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
        feed_dict[init_cell_output] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
        feed_dict[init_memory_output] = np.zeros([BATCH_SIZE, SLOT_SIZE])
        # Pass previous memory state.
        feed_dict[input_memory_state] = initial_memory_state #memory_state
            
    elif set_type_=="valid":
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = valid_batch[i]

        # Set previous weights of read and write heades.
        for i in range(SEQ_LENGTH):
            feed_dict[prev_read_weights_batch_seq[i]] = prev_rw_batch_seq[i]
            feed_dict[prev_write_weights_batch_seq[i]] = prev_ww_batch_seq[i]

        # Reset "initial" memory output.
        feed_dict[init_cell_state] = np.zeros([valid_size, HIDDEN_SIZE])
        feed_dict[init_cell_output] = np.zeros([valid_size, HIDDEN_SIZE])
        feed_dict[init_memory_output] = np.zeros([valid_size, SLOT_SIZE])
        # Pass previous memory state.
        feed_dict[input_memory_state] = initial_memory_state #memory_state

    else: # test
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = test_batch[i]
        
        # Set previous weights of read and write heades.
        for i in range(SEQ_LENGTH):
            feed_dict[prev_read_weights_batch_seq[i]] = prev_rw_batch_seq[i]
            feed_dict[prev_write_weights_batch_seq[i]] = prev_ww_batch_seq[i]

        # Reset "initial" memory output.
        feed_dict[init_cell_state] = np.zeros([test_size, HIDDEN_SIZE])
        feed_dict[init_cell_output] = np.zeros([test_size, HIDDEN_SIZE])
        feed_dict[init_memory_output] = np.zeros([test_size, SLOT_SIZE])
        # Pass previous memory state.
        feed_dict[input_memory_state] = initial_memory_state #memory_state
       
    return feed_dict

print("Feed_dict definition OK")

Feed_dict definition OK


### Session execution

In [16]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

print("Log dir CLEARED")

Log dir CLEARED


In [17]:

# Create session to execute graph.
sess=tf.InteractiveSession()

# Create summary writers, point them to LOG_DIR.
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize global variables.
tf.global_variables_initializer().run()
print('Variables initialized')

# Create initial previous read and update - full of zeros. 
# TODO: change BATCH_SIZE to 1 - reduce_mean all previous r/w weights. :]
prev_rw_batch_seq = list()
for i in range(SEQ_LENGTH):
    prev_rw_batch_seq.append(np.zeros([BATCH_SIZE, NUMBER_OF_SLOTS]))
prev_ww_batch_seq = list()
for i in range(SEQ_LENGTH):
    prev_ww_batch_seq.append(np.zeros([BATCH_SIZE, NUMBER_OF_SLOTS]))


# Initialize memory! random matrix 
initial_memory_state = sess.run(init_memory_state_op)
memory_state = initial_memory_state
print("memory_state =",memory_state.shape)
    
# Determine how long to perform the training and how often the test loss on validation batch will be computed. 
num_steps = 10000 #// (BATCH_SIZE*SEQ_LENGTH)
#num_steps = train_size // (BATCH_SIZE*SEQ_LENGTH)
summary_frequency = 10
#validation_frequency = 1000
print("Number of iterations per epoch =", num_steps)
for step in range(num_steps):
    memory_state, prev_rw_batch_seq, prev_ww_batch_seq, summaries, _, loss_, lr_ = sess.run([
        final_memory_state, read_weights_batch_seq, write_weights_batch_seq, merged_summaries, optimizer, loss, learning_rate],
        feed_dict=create_feed_dict("train"))
    # Every (100) steps collect statistics.
    if step % summary_frequency == 0:
        # Add summary.
        train_writer.add_summary(summaries, step*BATCH_SIZE*SEQ_LENGTH)
        train_writer.flush()
        print('Training set BPC at step %d: %0.5f learning rate: %f' % (step, loss_, lr_))

        #print("memory=\n", memory_)
        # Print loss from last batch.
    
    #if step % validation_frequency == 0:
    #    # Validation set BPC.
    #    print('=' * 80)
    #    print("Calculating BPC on validation set")
    #    v_summaries, v_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("valid"))
    #    print("Validation set BPC: %.5f" % v_loss)
    #    valid_writer.add_summary(v_summaries, step*BATCH_SIZE*SEQ_LENGTH)
    #    valid_writer.flush()
    # End of statistics collection

#for gv in grads_and_vars:
#    #print(str(sess.run(gv[0])) + " - " + gv[1].name)        
#    print(" - " + gv[1].name)        

# Test set BPC.
#print('=' * 80)tf.relu(
#print("Calculating BPC on test set")
#t_summary, t_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("test"))
#print("Final test set BPC: %.5f" % t_loss)
#test_writer.add_summary(t_summary, step*BATCH_SIZE*SEQ_LENGTH)
#test_writer.flush()
    
# Close writers and session.
train_writer.close()
valid_writer.close()
test_writer.close()
sess.close() 

Variables initialized
memory_state = (24, 32)
Number of iterations per epoch = 10000
Training set BPC at step 0: 4.15218 learning rate: 0.100000
Training set BPC at step 10: 3.51437 learning rate: 0.100000
Training set BPC at step 20: 3.77943 learning rate: 0.100000
Training set BPC at step 30: 2.14979 learning rate: 0.100000
Training set BPC at step 40: 4.11846 learning rate: 0.100000
Training set BPC at step 50: 4.59289 learning rate: 0.100000
Training set BPC at step 60: 3.67472 learning rate: 0.100000
Training set BPC at step 70: 3.25022 learning rate: 0.100000
Training set BPC at step 80: 3.83307 learning rate: 0.100000
Training set BPC at step 90: 3.18078 learning rate: 0.100000
Training set BPC at step 100: 3.36505 learning rate: 0.100000
Training set BPC at step 110: 4.52763 learning rate: 0.100000
Training set BPC at step 120: 4.35740 learning rate: 0.100000
Training set BPC at step 130: 3.72695 learning rate: 0.100000
Training set BPC at step 140: 4.44777 learning rate: 0.100

Training set BPC at step 1310: 2.62856 learning rate: 0.100000
Training set BPC at step 1320: 3.24039 learning rate: 0.100000
Training set BPC at step 1330: 2.75052 learning rate: 0.100000
Training set BPC at step 1340: 3.29039 learning rate: 0.100000
Training set BPC at step 1350: 3.16568 learning rate: 0.100000
Training set BPC at step 1360: 3.48480 learning rate: 0.100000
Training set BPC at step 1370: 3.72831 learning rate: 0.100000
Training set BPC at step 1380: 3.81411 learning rate: 0.100000
Training set BPC at step 1390: 2.42582 learning rate: 0.100000
Training set BPC at step 1400: 3.54692 learning rate: 0.100000
Training set BPC at step 1410: 2.95514 learning rate: 0.100000
Training set BPC at step 1420: 1.76769 learning rate: 0.100000
Training set BPC at step 1430: 2.84915 learning rate: 0.100000
Training set BPC at step 1440: 4.46948 learning rate: 0.100000
Training set BPC at step 1450: 3.18811 learning rate: 0.100000
Training set BPC at step 1460: 3.52750 learning rate: 0

Training set BPC at step 2620: 1.93804 learning rate: 0.100000
Training set BPC at step 2630: 2.36731 learning rate: 0.100000
Training set BPC at step 2640: 3.25975 learning rate: 0.100000
Training set BPC at step 2650: 3.15467 learning rate: 0.100000
Training set BPC at step 2660: 2.85688 learning rate: 0.100000
Training set BPC at step 2670: 3.43567 learning rate: 0.100000
Training set BPC at step 2680: 1.86897 learning rate: 0.100000
Training set BPC at step 2690: 2.74018 learning rate: 0.100000
Training set BPC at step 2700: 4.53217 learning rate: 0.100000
Training set BPC at step 2710: 2.55163 learning rate: 0.100000
Training set BPC at step 2720: 2.65383 learning rate: 0.100000
Training set BPC at step 2730: 2.57495 learning rate: 0.100000
Training set BPC at step 2740: 2.30701 learning rate: 0.100000
Training set BPC at step 2750: 2.16432 learning rate: 0.100000
Training set BPC at step 2760: 3.66994 learning rate: 0.100000
Training set BPC at step 2770: 3.02322 learning rate: 0

Training set BPC at step 3930: 4.29087 learning rate: 0.100000
Training set BPC at step 3940: 4.31454 learning rate: 0.100000
Training set BPC at step 3950: 2.21675 learning rate: 0.100000
Training set BPC at step 3960: 3.03609 learning rate: 0.100000
Training set BPC at step 3970: 4.34777 learning rate: 0.100000
Training set BPC at step 3980: 3.22621 learning rate: 0.100000
Training set BPC at step 3990: 3.97207 learning rate: 0.100000
Training set BPC at step 4000: 3.06827 learning rate: 0.100000
Training set BPC at step 4010: 3.08326 learning rate: 0.100000
Training set BPC at step 4020: 3.14258 learning rate: 0.100000
Training set BPC at step 4030: 3.23374 learning rate: 0.100000
Training set BPC at step 4040: 3.66680 learning rate: 0.100000
Training set BPC at step 4050: 2.61347 learning rate: 0.100000
Training set BPC at step 4060: 3.97335 learning rate: 0.100000
Training set BPC at step 4070: 4.02833 learning rate: 0.100000
Training set BPC at step 4080: 3.14882 learning rate: 0

Training set BPC at step 5240: 2.86678 learning rate: 0.090000
Training set BPC at step 5250: 3.26827 learning rate: 0.090000
Training set BPC at step 5260: 2.47822 learning rate: 0.090000
Training set BPC at step 5270: 2.67223 learning rate: 0.090000
Training set BPC at step 5280: 4.11507 learning rate: 0.090000
Training set BPC at step 5290: 2.53830 learning rate: 0.090000
Training set BPC at step 5300: 3.38717 learning rate: 0.090000
Training set BPC at step 5310: 3.82326 learning rate: 0.090000
Training set BPC at step 5320: 2.99095 learning rate: 0.090000
Training set BPC at step 5330: 3.59426 learning rate: 0.090000
Training set BPC at step 5340: 3.32720 learning rate: 0.090000
Training set BPC at step 5350: 3.49256 learning rate: 0.090000
Training set BPC at step 5360: 3.47224 learning rate: 0.090000
Training set BPC at step 5370: 3.37952 learning rate: 0.090000
Training set BPC at step 5380: 2.27499 learning rate: 0.090000
Training set BPC at step 5390: 4.15672 learning rate: 0

Training set BPC at step 6550: 2.77639 learning rate: 0.090000
Training set BPC at step 6560: 3.51817 learning rate: 0.090000
Training set BPC at step 6570: 4.64205 learning rate: 0.090000
Training set BPC at step 6580: 3.50338 learning rate: 0.090000
Training set BPC at step 6590: 3.11404 learning rate: 0.090000
Training set BPC at step 6600: 2.39848 learning rate: 0.090000
Training set BPC at step 6610: 3.12527 learning rate: 0.090000
Training set BPC at step 6620: 3.28953 learning rate: 0.090000
Training set BPC at step 6630: 3.94707 learning rate: 0.090000
Training set BPC at step 6640: 3.76982 learning rate: 0.090000
Training set BPC at step 6650: 3.43690 learning rate: 0.090000
Training set BPC at step 6660: 4.00544 learning rate: 0.090000
Training set BPC at step 6670: 3.13810 learning rate: 0.090000
Training set BPC at step 6680: 3.51408 learning rate: 0.090000
Training set BPC at step 6690: 4.08501 learning rate: 0.090000
Training set BPC at step 6700: 3.16192 learning rate: 0

Training set BPC at step 7860: 3.07524 learning rate: 0.090000
Training set BPC at step 7870: 3.25876 learning rate: 0.090000
Training set BPC at step 7880: 2.68752 learning rate: 0.090000
Training set BPC at step 7890: 3.06173 learning rate: 0.090000
Training set BPC at step 7900: 3.08483 learning rate: 0.090000
Training set BPC at step 7910: 2.70450 learning rate: 0.090000
Training set BPC at step 7920: 2.92528 learning rate: 0.090000
Training set BPC at step 7930: 4.06577 learning rate: 0.090000
Training set BPC at step 7940: 2.14163 learning rate: 0.090000
Training set BPC at step 7950: 3.39286 learning rate: 0.090000
Training set BPC at step 7960: 3.86660 learning rate: 0.090000
Training set BPC at step 7970: 2.76338 learning rate: 0.090000
Training set BPC at step 7980: 3.24396 learning rate: 0.090000
Training set BPC at step 7990: 4.42075 learning rate: 0.090000
Training set BPC at step 8000: 3.13675 learning rate: 0.090000
Training set BPC at step 8010: 3.80659 learning rate: 0

Training set BPC at step 9180: 2.83841 learning rate: 0.090000
Training set BPC at step 9190: 2.65743 learning rate: 0.090000
Training set BPC at step 9200: 4.74250 learning rate: 0.090000
Training set BPC at step 9210: 2.88191 learning rate: 0.090000
Training set BPC at step 9220: 2.78612 learning rate: 0.090000
Training set BPC at step 9230: 3.06098 learning rate: 0.090000
Training set BPC at step 9240: 3.98351 learning rate: 0.090000
Training set BPC at step 9250: 3.29194 learning rate: 0.090000
Training set BPC at step 9260: 3.39108 learning rate: 0.090000
Training set BPC at step 9270: 3.42525 learning rate: 0.090000
Training set BPC at step 9280: 2.52103 learning rate: 0.090000
Training set BPC at step 9290: 3.18321 learning rate: 0.090000
Training set BPC at step 9300: 2.57653 learning rate: 0.090000
Training set BPC at step 9310: 2.82992 learning rate: 0.090000
Training set BPC at step 9320: 2.34994 learning rate: 0.090000
Training set BPC at step 9330: 3.67082 learning rate: 0